In [ ]:
import os
import pandas as pd
from matplotlib.pyplot import *
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal, stats
%matplotlib inline

In [ ]:
working_dir = os.getcwd()
data_dir = os.path.join(working_dir, "..", "data")

In [ ]:
"""Load Pierre Raw Data"""
pierre = pd.read_csv(os.path.join(data_dir, "data_pierre_1.csv"))
pierre_t = np.array(pierre['timestamps'])
pierre_1 = np.array(pierre["Unnamed: 1"])
pierre_2 = np.array(pierre["Unnamed: 2"])
pierre_3 = np.array(pierre["Unnamed: 3"])
pierre_4 = np.array(pierre["Unnamed: 4"])

In [ ]:
"""Load Gaby Raw Data"""
gaby = pd.read_csv(os.path.join(data_dir, "data_gaby.csv"))
gaby_t = np.array(gaby['timestamps'])
gaby_1 = np.array(gaby["Unnamed: 1"])
gaby_2 = np.array(gaby["Unnamed: 2"])
gaby_3 = np.array(gaby["Unnamed: 3"])
gaby_4 = np.array(gaby["Unnamed: 4"])

In [ ]:
print("gaby's start time is", pierre_t[0] - gaby_t[0], "earlier than pierre")
print("gaby's end time is", gaby_t[-1] - pierre_t[-1], "later than pierre")
print("need to trim gaby on both ends to fit pierre")

In [ ]:
"""Trim gaby's start time to match pierre'"""
gaby_t = gaby_t[np.where(gaby_t >= pierre_t[0])]
gaby_1 = gaby_1[np.where(gaby_t >= pierre_t[0])]
gaby_2 = gaby_2[np.where(gaby_t >= pierre_t[0])]
gaby_3 = gaby_3[np.where(gaby_t >= pierre_t[0])]
gaby_4 = gaby_4[np.where(gaby_t >= pierre_t[0])]

In [ ]:
print("gaby's start time is", pierre_t[0] - gaby_t[0], "earlier than pierre")
print("gaby's end time is", gaby_t[-1] - pierre_t[-1], "later than pierre")

In [ ]:
"""Trim gaby's end time to match pierre"""
diverging_index = np.where(gaby_t >= pierre_t[-1])[0][0]
gaby_t = gaby_t[:diverging_index]
gaby_1 = gaby_1[:diverging_index]
gaby_2 = gaby_2[:diverging_index]
gaby_3 = gaby_3[:diverging_index]
gaby_4 = gaby_4[:diverging_index]

In [ ]:
print("gaby's start time is", pierre_t[0] - gaby_t[0], "earlier than pierre")
print("gaby's end time is", gaby_t[-1] - pierre_t[-1], "later than pierre")

In [ ]:
gaby_fs = 1/np.mean(np.diff(gaby_t))
print("gaby's sampling frequency is", gaby_fs)
pierre_fs = 1/np.mean(np.diff(pierre_t))
print("pierre's sampling frequency is", pierre_fs)
print("Therefore, pierre has more samples within the same timecourse.")
print("pierre has", len(pierre_t) - len(gaby_t), "more samples than gaby within same timecourse.")
print("Need to downsample pierre to have same number of data points as gaby.")

In [ ]:
"""Resampling pierre to gaby's length"""
pierre_1 = signal.resample(pierre_1, len(gaby_t))
pierre_2 = signal.resample(pierre_2, len(gaby_t))
pierre_3 = signal.resample(pierre_3, len(gaby_t))
pierre_4 = signal.resample(pierre_4, len(gaby_t))

In [ ]:
print("pierre has", len(pierre_1) - len(gaby_1), "more samples than gaby within same timecourse.")

In [ ]:
fs = gaby_fs

In [ ]:
pierre_data = np.concatenate((pierre_1.reshape(-1,1), pierre_2.reshape(-1,1), pierre_3.reshape(-1,1), pierre_4.reshape(-1,1)), axis=1)

In [ ]:
gaby_data = np.concatenate((gaby_1.reshape(-1,1), gaby_2.reshape(-1,1), gaby_3.reshape(-1,1), gaby_4.reshape(-1,1)), axis=1)

In [ ]:
## filter signal to remove noise
b, a = signal.butter(2, (2/(fs/2), 30/(fs/2)), btype='bandpass')
pierre_smoothed = signal.filtfilt(b, a, pierre_data, axis=0)
gaby_smoothed = signal.filtfilt(b, a, gaby_data, axis=0)

In [ ]:
window = 200
step = 25
corr = []
times = []

#advance window of 200 samples by 25 samples each time
## take correlation between signals across each sample
for start in np.arange(0, len(pierre_smoothed), step):
    end = start + window
    w1 = pierre_smoothed[start:end]
    w2 = gaby_smoothed[start:end]
    
    ## average the correlation across each channel
    r = 0
    for c in range(w1.shape[1]): #for each electrode
        r += stats.pearsonr(w1[:, c], w2[:, c])[0] #correlate that electrode between gaby and pierre
    r /= w1.shape[1] #average correlation across electrodes
    
    mid = (start+end)/2 # middle sample
    t = mid / fs # convert middle sample to time
    
    times.append(t)
    corr.append(r)

times = np.array(times)
corr = np.array(corr)

In [ ]:
np.savetxt("times.txt", times)
np.savetxt("corr.txt", corr)

In [ ]:
figure(figsize=(14,4))
plot(times, corr)
xlabel('Time (s)')
ylabel('Correlation')
_ = title('Correlation across brains')

In [ ]:
"""Syncing to video content"""
start_white = 1492292390.07327
start_black = 1492292391.07712
end_white = 1492297704.94143
end_black = 1492297705.94569

In [ ]:
print("Brain recordings lasted for", int(times[-1]), "seconds")

In [ ]:
print("Brain recordings started", pierre_t[0] - start_black, "seconds after black screen")

In [ ]:
print("Brain recordings ended", end_black - pierre_t[-1], "seconds before black screen")

In [ ]:
#need to run this separately
"""
===========
MovieWriter
===========

This example uses a MovieWriter directly to grab individual frames and write
them to a file. This avoids any event loop integration, but has the advantage
of working with even the Agg backend. This is not recommended for use in an
interactive setting.

"""
# -*- noplot -*-

import numpy as np
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import matplotlib.animation as manimation

FFMpegWriter = manimation.writers['ffmpeg']
metadata = dict(title='Movie Test', artist='Matplotlib',
                comment='Movie support!')
writer = FFMpegWriter(fps=15, metadata=metadata)

from matplotlib.figure import Figure
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

times = np.loadtxt("times.txt")
corr = np.loadtxt("corr.txt")
norm = plt.Normalize()
colors = plt.cm.jet(norm(corr))

fig = plt.figure()
l, = plt.plot([], [], 'k-o', markersize=10)
ax = plt.gca()

with writer.saving(fig, "writer_test_color.mp4", 100):
    for i in range(len(times)):
        plt.xlim(times[i]-10, times[i]+10)
        plt.ylim(-1, 1)
        ax.set_axis_bgcolor(colors[i])
        l.set_data(times[i], corr[i])
        writer.grab_frame()